In [5]:
!pip show transformers torch

Name: transformers
Version: 4.57.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /Users/bruno/opt/anaconda3/envs/env-misc/lib/python3.11/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: sentence-transformers
---
Name: torch
Version: 2.2.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /Users/bruno/opt/anaconda3/envs/env-misc/lib/python3.11/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accele

In [8]:
!pip install -U torch 

# 4.7 — Transformers (Parte 7): GPT e Modelos Autoregressivos (*decoder-only*)

**Objetivos da aula**
- Entender o paradigma *decoder-only* e como ele difere do *encoder-only* (BERT).
- Compreender a **modelagem de linguagem autoregressiva**: prever o próximo token.
- Explorar a **família GPT** (GPT, GPT-2, GPT-3, GPT-4 e open-source “GPT-like”).
- Fazer **inferência** com GPT-2 (Hugging Face) e **controlar a geração** (temperatura, top-k, top-p).
- Realizar um **mini fine-tuning** para *causal language modeling* (CLM).
- Medir *loss* e **perplexity** e discutir limitações, riscos e boas práticas.

**Pré-requisitos**
- Conceitos básicos de Transformers, *self-attention* e *positional encoding*.
- Familiaridade com BERT e fine-tuning usando `transformers` (Hugging Face).

## 1) Relembrando... De BERT ao GPT: a virada dos modelos generativos

Os modelos baseados em **Transformers** revolucionaram o processamento de linguagem natural ao introduzirem a ideia de atenção paralela entre todos os tokens de uma sequência.  
Entretanto, as primeiras grandes implementações — como **BERT** (2018) — tinham um foco principal em **compreensão** de texto, e não em **geração**.

---

### O papel do BERT e suas limitações

O **BERT (Bidirectional Encoder Representations from Transformers)** foi projetado como um **modelo encoder-only**:
- Treinado com o objetivo de **Masked Language Modeling (MLM)**, em que partes da sentença são mascaradas e o modelo precisa prever as palavras ocultas.  
- Seu aprendizado é **bidirecional**, ou seja, cada token “enxerga” tanto o passado quanto o futuro da sequência.  
- Essa característica torna o BERT excelente para **tarefas de interpretação**, como:
  - classificação de textos,  
  - análise de sentimentos,  
  - reconhecimento de entidades nomeadas,  
  - perguntas e respostas (*extractive QA*).

Apesar de sua enorme contribuição, o BERT apresentava **restrições claras**:
- Não podia ser usado naturalmente para **gerar texto contínuo** (como escrever, traduzir ou responder perguntas abertas).  
- O processo de *masking* dificultava o uso em aplicações **autoregressivas** — ou seja, prever o próximo token em tempo real.  
- Cada tarefa exigia um **cabeçalho (head) especializado**, dificultando a generalização e o reuso direto do modelo em contextos diferentes.

---

### A motivação para os modelos GPT

O **GPT (Generative Pre-trained Transformer)** nasceu como resposta direta a essas limitações.  
Enquanto o BERT se baseava em **atenção bidirecional (encoder)**, o GPT inverteu a lógica e apostou em **atenção causal unidirecional (decoder)**.

A mudança de paradigma trouxe ganhos decisivos:
- O modelo passou a **gerar texto palavra a palavra**, aprendendo a prever o **próximo token** em vez de preencher lacunas.  
- Isso o tornou naturalmente adequado para **tarefas abertas e criativas**, como diálogo, resumo, tradução e redação assistida.  
- A mesma arquitetura poderia agora **aprender uma variedade de tarefas supervisionadas** apenas reformatando o texto de entrada — sem alterar a estrutura interna.

Em resumo:

| Aspecto | BERT (Encoder-only) | GPT (Decoder-only) |
|:--|:--|:--|
| Tipo de atenção | Bidirecional | Causal (unidirecional) |
| Objetivo de treino | Prever tokens mascarados (MLM) | Prever o próximo token (LM) |
| Tipo de tarefa | Compreensão (análise, classificação) | Geração (continuação, síntese) |
| Direção da informação | Passado + futuro | Somente passado |
| Saída típica | Vetores de representação | Texto gerado |


O **GPT surge como uma evolução natural do Transformer**, voltando às suas raízes *decoder-only*.  
Ele herda a eficiência do mecanismo de atenção introduzido no BERT, mas a aplica de forma **autoregressiva**, permitindo **geração coerente, contextual e adaptável**.  
Esse movimento marca a transição dos modelos **discriminativos** (como BERT) para os **generativos**, pavimentando o caminho para a era dos **Large Language Models** (GPT-2, GPT-3, ChatGPT, LLaMA, Claude, Gemini, entre outros).

### 1.1) Encoder-only (BERT) vs Decoder-only (GPT)

**BERT (encoder-only)**:
- Atenção bidirecional (*masked language modeling*, MLM).
- Aprende representações profundas para **compreensão** (classificação, NER, QA com *span extraction*).
- Treinamento: prever **tokens mascarados** + (opcional) *Next Sentence Prediction (NSP)*.

**GPT (decoder-only)**:
- Atenção **causal** (unidirecional): cada posição só “vê” tokens anteriores.
- Treinamento **autoregressivo**: prever o **próximo token**.
- Excelente para **geração** de texto, continuação de prompts e *in-context learning*.

Em termos de probabilidade:
$$
P(w_1,\dots,w_T) = \prod_{t=1}^{T} P(w_t \mid w_1,\dots,w_{t-1})
$$
No GPT, modelamos $P(w_t \mid w_{<t})$ e aplicamos *teacher forcing* durante o treino (prever o próximo token com base no prefixo correto).

## 2) Arquitetura GPT (Decoder-only) e Máscara Causal

**Bloco GPT:**
- *Embedding* de tokens + **positional embeddings** aprendíveis.
- Pilha de camadas *decoder*: 
  - *Masked self-attention* (atenção causal com máscara triangular superior).
  - *Feed-forward network* (MLP com GELU).
  - *Residual connections* + *LayerNorm* (normalização por camada).
- Projeção final para o vocabulário (logits).

**Máscara causal**: impede “vazamento” de informações do futuro.
- Se o token está na posição *t*, ele só pode atender (atender = *attend*) às posições $\leq t$\)$.
- Implementada como uma matriz triangular **superior** com $-\infty$ onde não pode atender.

## 2.1) Arquitetura do GPT-1 e fine-tuning supervisionado

A figura abaixo ilustra a **arquitetura original do GPT-1** (2018) e as formas como o modelo foi ajustado para diferentes tarefas supervisionadas.  
O GPT-1 foi o primeiro modelo a demonstrar que um **Transformer *decoder-only***, pré-treinado com **modelagem de linguagem autoregressiva**, podia ser reutilizado para várias tarefas de NLP — bastando estruturar a entrada textual adequadamente e adicionar uma **camada linear de classificação** sobre o último token.

<p align="center">
<img src="https://raw.githubusercontent.com/bmnogueira-ufms/TopicosIA-2025-02/main/images/gpt_arquitetura.png" width="50%%">
  Fonte: Kamath et al. (2024)
</p>

**Arquitetura do GPT:**
- O modelo GPT-1 consiste em **12 blocos *decoder*** empilhados, cada um contendo:
  - *Masked Multi-Head Self-Attention* (atenção causal),
  - camadas *Feed-Forward* com ativação GELU,
  - *Layer Normalization* e conexões residuais.
- Na base, os tokens são convertidos em vetores de embedding somando:
  - **Embeddings de palavras** (aprendidos),
  - **Embeddings de posição** (também aprendidos).

**Fine-tuning supervisionado:**
Durante o ajuste para tarefas específicas, o texto de entrada é formatado conforme a natureza da tarefa:

| Tarefa | Estrutura de entrada | Saída gerada |
|:--|:--|:--|
| **Classificação** | `Start ⟶ Text ⟶ Extract` | Predição linear no token `Extract` |
| **Entailment / Inferência textual** | `Start ⟶ Premise ⟶ Delim ⟶ Hypothesis ⟶ Extract` | Classificação no token final |
| **Similaridade textual** | `Start ⟶ Text₁ ⟶ Delim ⟶ Text₂ ⟶ Extract` | Combinação de embeddings antes do `Linear` |
| **Múltipla escolha** | `Start ⟶ Context ⟶ Delim ⟶ Answer₁…N ⟶ Extract` | Cada resposta passa por uma *linear layer*, e a pontuação é normalizada via *softmax* |

> **Ideia central:** o modelo não precisa de arquitetura diferente para cada tarefa — apenas de um *prompt* ou sequência de entrada bem estruturada.  
> Esse conceito inspirou o *paradigma de instruções* e o *prompt-based learning* que evoluíram nas famílias GPT-2, GPT-3 e ChatGPT.

#### Tokens especiais

O GPT-1 utiliza três tipos de marcadores auxiliares:

| Token | Significado | Função |
|:--|:--|:--|
| **Start** | Marcador inicial da sequência | Indica o início do exemplo e ajuda o modelo a delimitar o contexto. |
| **Delim** | Delimitador | Separa partes distintas da entrada (por exemplo, “premissa” e “hipótese”). |
| **Extract** | Token de extração | Indica o ponto em que a predição deve ser feita (posição do *head* linear). |

A **camada linear final** (`Linear`) é conectada ao vetor de saída correspondente ao token **Extract**, sobre o qual se aplica uma operação de *softmax* para gerar as probabilidades das classes.

O GPT-1 não precisava de cabeçalhos nem módulos especializados para cada tarefa.  
Ele apenas aprendia **a continuar uma sequência textual estruturada**.  
Esse conceito deu origem ao **prompt-based learning** e, mais tarde, ao **in-context learning** dos modelos GPT-3 e ChatGPT.

#### Task Classifier e Test Prediction

Durante o **fine-tuning supervisionado**, o modelo pré-treinado precisa se adaptar a uma tarefa específica.  
Para isso, adicionamos e utilizamos duas componentes fundamentais: o **task classifier** e o **test prediction**.

---

##### Task Classifier

O **task classifier** é uma camada adicional (geralmente linear + *softmax*) conectada à saída do modelo Transformer.  
Seu papel é transformar as **representações contextuais** produzidas pelo modelo em **predições específicas da tarefa**.

| Exemplo de tarefa | Entrada | Saída do *task classifier* |
|:--|:--|:--|
| Classificação de sentimento | Texto completo | Positivo / Negativo |
| NER (Reconhecimento de Entidades) | Sentença tokenizada | Tag por token (`B-PER`, `I-ORG`, etc.) |
| Inferência textual | Premissa + Hipótese | Entailment / Contradiction / Neutral |

No **BERT**, essa camada é aplicada sobre o vetor do token `[CLS]`.  
No **GPT**, como o modelo já é autoregressivo, a saída final do último token (*Extract* ou *último hidden state*) é usada como entrada para o classificador.

---

##### Test Prediction

O termo **test prediction** refere-se à **etapa de inferência ou avaliação**, quando o modelo — já treinado — é aplicado ao conjunto de teste.

Durante essa fase:
1. As entradas são processadas pelo Transformer sem atualização de pesos.  
2. O *task classifier* transforma as ativações do modelo em rótulos ou probabilidades.  
3. As predições são comparadas com as verdadeiras para calcular métricas como *accuracy*, *F1-score* ou *perplexity*.

---

##### Diferença essencial

| Etapa | Função | Ocorrência |
|:--|:--|:--|
| **Task Classifier** | Transforma as representações internas em saídas específicas da tarefa | Durante o *fine-tuning* |
| **Test Prediction** | Gera as predições finais sobre dados de teste | Durante a avaliação (inferência) |

Em resumo, o **task classifier** é o mecanismo de saída que “traduz” o conhecimento do modelo,  
enquanto o **test prediction** é o momento em que verificamos **o quanto ele aprendeu de fato**.

---

### 2.2 Procedimento de treinamento (GPT-1)

O GPT-1 foi treinado em **duas fases**:

1. **Pré-treinamento não supervisionado** (*unsupervised pre-training*):  
   - O modelo aprende a prever o próximo token em uma sequência de texto grande, segundo o objetivo de *Language Modeling*:
     $$
     \mathcal{L}_1(U) = \sum_i \log P(u_i \mid u_{i-k}, ..., u_{i-1}; \Theta)
     $$
     onde $U = $u_1, ..., u_n$$ é o corpus não supervisionado, $k$ o tamanho do contexto e $\Theta$ os parâmetros do modelo.

2. **Fine-tuning supervisionado** (*supervised fine-tuning*):  
   - Após o pré-treinamento, o modelo é adaptado a tarefas específicas com dados rotulados $C = (x, y)$:
     $$
     P(y \mid x_1, ..., x_m) = \text{softmax}(h_m W_y)
     $$
     e o objetivo supervisionado é:
     $$
     \mathcal{L}_2(C) = \sum_{(x,y)} \log P(y \mid x_1, ..., x_m)
     $$
   - Durante o fine-tuning, o uso do *Language Modeling* como objetivo auxiliar melhora a generalização:
     $$
     \mathcal{L}_3 = \mathcal{L}_2(C) + \lambda \times \mathcal{L}_1(U)
     $$

> **Conclusão:** O GPT-1 demonstrou que o *pre-training + fine-tuning* em um modelo gerativo era suficiente para atingir *transfer learning* em várias tarefas de NLP — conceito que viria a dominar o campo nos anos seguintes.

---

### 2.3 GPT-2 — Escalando a arquitetura

O **GPT-2** (2019) manteve o mesmo design arquitetural, mas:
- Expandiu de **117M para 1.5B parâmetros** (≈10× maior).
- Foi treinado em um corpus mais amplo e diverso (*WebText*), com foco apenas em *unsupervised language modeling*, sem fine-tuning supervisionado.

Formalmente, o treinamento de GPT-2 visa estimar:
$$
p(\text{output} \mid \text{input}, \text{task})
$$
onde o modelo é condicionado tanto ao texto quanto a instruções que definem a tarefa (“traduza para o francês”, “resuma o texto”, etc.).  
Isso permitiu que o modelo aprendesse a **generalizar entre múltiplos domínios**, aproximando-se da ideia de *multi-task learning sem rótulos explícitos*.

**Principais resultados e observações:**
- Mostrou que **aumentar parâmetros e dados** melhora desempenho quase linearmente.  
- O GPT-2 **subajustou** (*underfitted*) o dataset WebText — indicando que ainda não havia saturação de capacidade.  
- Superou *benchmarks* em **7 de 8 datasets de language modeling**, incluindo grande salto de desempenho no **LAMBADA** (perplexity 99.8 → 8.6).

> **Conclusão:** GPT-2 provou empiricamente o valor do *scaling law* — quanto mais parâmetros e dados, melhor a generalização e menor a perplexity.

---

### 2.4 GPT-3 — O paradigma do *in-context learning*

O **GPT-3** (2020) manteve a base arquitetural dos anteriores, mas:
- Escalou para **175 bilhões de parâmetros**;
- Ampliou a **janela de contexto** (2048 tokens);
- Foi treinado em **corpus muito maior e mais variado**, sem fine-tuning supervisionado.

Em vez de ajustar pesos para cada tarefa, o GPT-3 introduziu o **aprendizado no contexto** (*in-context learning*), operando em três modos:

| Tipo | Descrição | Exemplo |
|:--|:--|:--|
| **Zero-shot** | O modelo realiza a tarefa apenas com base na instrução textual. | “Traduza para o francês: The book is new.” |
| **One-shot** | Um exemplo é fornecido antes da tarefa. | “Inglês → Francês: cat → chat. Dog → ?” |
| **Few-shot** | Vários exemplos são dados no prompt. | Lista de pares de tradução antes da nova frase. |

O GPT-3 demonstrou desempenho *state-of-the-art* em **múltiplas tarefas sem fine-tuning**, incluindo:
- **QA** (perguntas e respostas);
- **Tradução**;
- **Compreensão de leitura**;
- **Raciocínio básico e aritmética textual**.

Porém, ainda apresentou limitações:
- **Coerência em textos longos** e **repetição excessiva**;
- Dificuldade em *inference tasks* (entailment, NLI);
- Custo computacional elevado e pouca interpretabilidade.

> **Risco e impacto social:** GPT-3 introduziu o desafio da geração *human-like* e o potencial de uso indevido (fake news, phishing, etc.), destacando a importância de segurança e regulação de LLMs.

---

### 2.5 Variantes e sucessores modernos

A filosofia *decoder-only* do GPT influenciou uma nova geração de modelos de larga escala:

| Modelo | Parâmetros | Treinamento / Dataset | Características principais |
|:--|:--|:--|:--|
| **Gopher (DeepMind, 2021)** | 280B | 300B tokens (MassiveText) | Superou GPT-3 em 81% de 100 tarefas. |
| **Chinchilla (DeepMind, 2022)** | 70B | 1.4T tokens | Compute-óptimo, menor custo e melhor desempenho. |
| **LLaMA (Meta, 2023)** | 8B–70B | Dados web, código, Wikipedia | Modelos abertos, fine-tuning + RLHF para diálogo. |
| **Claude 3 (Anthropic, 2023)** | 20B–2T | Dados multimodais | Modelos Opus, Sonnet e Haiku: balanceiam custo e velocidade. |
| **Command R (Cohere, 2024)** | 35B | 128k contexto | Otimizado para RAG e uso de ferramentas externas. |
| **Gemma (Google, 2024)** | 2B–7B | 6T tokens | Foco em segurança, raciocínio e desempenho multilingue. |

> **Tendência geral:** reduzir custos mantendo desempenho → *compute-optimal training*, *long-context modeling* e *instrução supervisionada* (SFT + RLHF).

---

### 2.6 Principais lições aprendidas

1. **GPT-1** provou que o *language modeling pre-training* é um poderoso mecanismo de transferência.  
2. **GPT-2** mostrou que escalar parâmetros e dados leva a ganhos contínuos e robustos — sem saturação aparente.  
3. **GPT-3** inaugurou o *in-context learning*, tornando possível generalização para novas tarefas apenas via *prompt*.  
4. **Modelos subsequentes** (Gopher, Chinchilla, LLaMA, Claude, Command R, Gemma) consolidaram essa linha, equilibrando tamanho, custo e desempenho.  
5. A evolução da série GPT definiu o paradigma atual dos **LLMs autoregressivos**, no qual *prompt design*, *instruções* e *context windows ampliadas* substituem o fine-tuning clássico.

## 3) Família GPT (linha do tempo resumida)

| Modelo  | Ano  | Parâmetros (maior variante) | Treino / Dados               | Destaques                                       |
|---------|------|------------------------------|------------------------------|--------------------------------------------------|
| GPT     | 2018 | ~117M                        | BooksCorpus                  | Prova de conceito de *decoder-only* autoregressivo |
| GPT-2   | 2019 | 1.5B                         | WebText                      | Geração coerente; mostrou o poder do *scaling*   |
| GPT-3   | 2020 | 175B                         | Diversos (web-scale)         | *Few-shot*, *in-context learning*                |
| GPT-3.5 | 2022 | —                            | + Instruções                 | Basis para “ChatGPT” (instruções + RLHF)         |
| GPT-4   | 2023 | —                            | Multimodal                   | Raciocínio melhor, contexto maior                |

**Observações**:
- O “salto” de desempenho surge de **mais dados, mais parâmetros e melhores instruções** (técnicas como RLHF, *system prompts*, *SFT* com *instruction-tuning*).
- Em open-source, há modelos “GPT-like” (p.ex., GPT-Neo, GPT-J, LLaMA-based, MPT) que seguem o **paradigma decoder-only**.

## 4) Ambiente e bibliotecas

Vamos usar a biblioteca **Hugging Face Transformers** para:
- Carregar *tokenizer* e modelo **GPT-2**.
- Fazer **inferência** (geração) controlando temperatura, *top-k*, *top-p*.
- Rodar um **mini fine-tuning** para CLM (Causal Language Modeling).

> **Dica**: para execução mais rápida, use a variante `gpt2` (pequena). Se a máquina for modesta, reduza *max_length*, *batch_size* e *block_size*.

In [2]:
# %pip install -q transformers datasets accelerate
import torch
from transformers import (
    GPT2LMHeadModel, GPT2TokenizerFast,
    set_seed, DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from datasets import Dataset
import math

device = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(42)
device

'cpu'

In [9]:
# model_name = "gpt2"
model_name = "pierreguillou/gpt2-small-portuguese"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

# GPT-2 não tem token de padding por padrão; definimos para evitar avisos no Trainer
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name, use_safetensors=True)
model.to(device)

# Inspeção rápida da tokenização
text = "Inteligência Artificial está transformando a educação."
enc = tokenizer(text, return_tensors="pt")
print("IDs:", enc["input_ids"][0][:10], "... tam =", enc["input_ids"].shape[-1])
print("Decodificado:", tokenizer.decode(enc["input_ids"][0]))

IDs: tensor([18599, 15960,   861, 46387,   827, 13772,   259,  3455,    14]) ... tam = 9
Decodificado: Inteligência Artificial está transformando a educação.


## 5) Geração autoregressiva: greedy vs amostragem

**Greedy**: escolhe sempre o token de maior probabilidade.
- Pro: mais determinístico.
- Contra: pode “entravar” e repetir.

**Amostragem com temperatura e *top-k*/*top-p***:
- `temperature`: suaviza as probabilidades (↑ = mais aleatório).
- `top_k`: restringe aos *k* tokens mais prováveis.
- `top_p` (*nucleus sampling*): restringe ao menor conjunto de tokens cuja soma de probabilidades ≥ *p*.

> Boas práticas: combine `temperature` com `top_p` (ou `top_k`) para controlar fluência e diversidade.

In [10]:
def generate(prompt, **gen_kwargs):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, **gen_kwargs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "A inteligência artificial nas escolas do Brasil"

print("=== Greedy ===")
print(generate(prompt, max_length=60, do_sample=False))

print("\n=== Sampling (temp=0.8, top_k=50, top_p=0.9) ===")
print(generate(prompt, max_length=60, do_sample=True, temperature=0.8, top_k=50, top_p=0.9))

=== Greedy ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A inteligência artificial nas escolas do Brasil, como a Escola de Aperfeiçoamento de Oficiais (ECAO), a Escola de Aperfeiçoamento de Oficiais (ECAO), a Escola de Aperfeiçoamento de Oficiais (ECAO), a Escola de Aperfeiçoamento de Oficiais (ECAO), a Escola de

=== Sampling (temp=0.8, top_k=50, top_p=0.9) ===
A inteligência artificial nas escolas do Brasil, no Brasil, nos Estados Unidos, na África, na Ásia, na África, na Ásia e no mundo, é um dos pilares da estratégia militar de inteligência artificial que está sendo desenvolvida nos Estados Unidos.

Os analistas militares da inteligência artificial no Brasil, na


In [11]:
prompt = "Resumo: As redes neurais transformadoras permitem"
temperatures = [0.3, 0.7, 1.0]
top_ps = [0.8, 0.9, 0.95]

for T in temperatures:
    for P in top_ps:
        print(f"\n--- temperature={T}, top_p={P} ---")
        print(generate(prompt, max_length=80, do_sample=True, temperature=T, top_p=P))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- temperature=0.3, top_p=0.8 ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Resumo: As redes neurais transformadoras permitem que os neurônios sejam movidos para um local específico de propagação, e o tempo de propagação é medido em tempo.

O tempo de propagação é medido em tempo por meio de um gráfico de tempo. O tempo de propagação é medido em tempo por meio de um gráfico de tempo.

O tempo de propagação é medido em tempo por meio de um gráfico

--- temperature=0.3, top_p=0.9 ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Resumo: As redes neurais transformadoras permitem que as redes neurais locais e locais tenham uma melhor capacidade de processar informações e aprender padrões de aprendizado em tempo real.

As redes neurais locais podem ser treinadas em tempo real ou simultaneidade, com diferentes níveis de detalhe, permitindo a construção de padrões de aprendizado que são mais complexos do que os padrões de aprendizado de um sistema. Além disso,

--- temperature=1.0, top_p=0.8 ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Resumo: As redes neurais transformadoras permitem que a rede atualize seu estado de saúde ao longo de todo o ciclo.


As redes neurais são projetadas para armazenar informação de forma precisa para que o computador tenha tempo suficiente para aprender e reproduzir o estado de uma rede. Como a informação é transferida através de um processo de aprendizado, o sistema de decisão de qual informação a ser transmitida é

--- temperature=1.0, top_p=0.9 ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Resumo: As redes neurais transformadoras permitem que uma rede neural que não está completamente mapeada de uma rede neural, como o cérebro, experimentasse os estados alterados por algum estímulo. As redes neurais não somente podem ser utilizadas para criar um processo que induz uma resposta, como também podem alterar o padrão de resposta para outros processos cognitivos (por exemplo, mudanças na temperatura, distância, etc.).

--- temperature=1.0, top_p=0.95 ---
Resumo: As redes neurais transformadoras permitem aos humanos detectar, interagir, orientar a ação, organizar a atividade, armazenar informações para serem usadas na forma de memória e processar os dados. A tecnologia de redes neurais artificiais foi desenvolvida pela Universidade Federal de Santa Catarina para resolver os desafios que os humanos enfrentam diante da comunicação de informações. O cérebro humano utiliza a tecnologia de Redes neurais como um meio de interagir com


## 6) Fine-tuning (CLM) com GPT-2 em corpus pequeno

**Objetivo**: adaptar o GPT-2 para um *estilo* ou *domínio* (ex.: resumos, ementas, notícias curtas), usando **Causal Language Modeling**:
- Treinaremos para prever o **próximo token** (mlm=False).
- Uniremos textos e criaremos *blocos* de tamanho fixo (*block_size*), como janelas deslizantes.

> **Atenção**: com corpus pequeno, espere **overfitting** e ganho limitado. Use isso como **demonstração didática**.

In [12]:
texts = [
    "Resumo: Este trabalho apresenta uma introdução a modelos transformer aplicados a NLP.",
    "Discussão: A arquitetura decoder-only baseia-se em atenção causal e geração autoregressiva.",
    "Resultados: Observamos melhora na coerência dos textos com ajuste de temperatura e top-p.",
    "Aplicações: Educação, atendimento automatizado, redação assistida e sumarização.",
    "Limitações: Viés, alucinação e sensibilidade ao prompt exigem avaliação cuidadosa."
] * 50  # replicamos para ter um pouco mais de dados

raw_ds = Dataset.from_dict({"text": texts})

# Tokenização
block_size = 128  # diminua se estiver com pouca memória
def tokenize(examples):
    return tokenizer(examples["text"])

tokenized = raw_ds.map(tokenize, batched=True, remove_columns=["text"])

# Juntar tokens em blocos contíguos (causal LM)
def group_texts(examples):
    # concatena e corta em blocos de block_size
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_len = len(concatenated["input_ids"])
    total_len = (total_len // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_len, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tokenized.map(group_texts, batched=True)
lm_ds

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 35
})

In [14]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="runs-gpt2-clm",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.05,
    logging_steps=20,
    save_strategy="no",
    fp16=torch.cuda.is_available(),
)
# training_args = TrainingArguments(
#     output_dir="runs-gpt2-clm",
#     overwrite_output_dir=True,
#     num_train_epochs=1,          # didático; aumente com GPU/dados
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=2,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     warmup_ratio=0.05,
#     logging_steps=20,
#     save_strategy="no",
#     evaluation_strategy="no",    # poderia separar um split de validação para eval
#     fp16=torch.cuda.is_available(),
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_ds,
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer

/var/folders/4g/hm9d3pn57nv1p17qlv8k47mc0000gn/T/ipykernel_63881/2718657328.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
train_result = trainer.train()
train_result

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 0, 'bos_token_id': 0, 'pad_token_id': 0}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=9, training_loss=3.501142289903429, metrics={'train_runtime': 60.522, 'train_samples_per_second': 0.578, 'train_steps_per_second': 0.149, 'total_flos': 2286305280000.0, 'train_loss': 3.501142289903429, 'epoch': 1.0})

In [16]:
# Avaliação simples no próprio treino (didático): calcula perplexity a partir do loss
final_loss = train_result.training_loss
perplexity = math.exp(final_loss) if final_loss < 20 else float("inf")
print(f"Loss final (train): {final_loss:.4f}")
print(f"Perplexity aproximada: {perplexity:.2f}")

Loss final (train): 3.5011
Perplexity aproximada: 33.15


In [ ]:
prompt = "Resumo:"
# print("=== Antes do ajuste (baseline pode variar) ===")
# model = GPT2LMHeadModel.from_pretrained(model_name)
# model.to(device)
# print(generate(prompt, max_length=60, do_sample=True, temperature=0.8, top_p=0.9))
print("\n=== Depois do fine-tuning ===")
print(generate(prompt, max_length=60, do_sample=True, temperature=0.8, top_p=0.9))

## 7) Boas práticas, riscos e limitações

- **Overfitting** com pouco dado: o modelo memoriza padrões; use *early stopping*, *regularização*, *data augmentation*.
- **Viés e alucinação**: modelos autoregressivos podem inventar fatos; sempre valide saídas.
- **Controle de geração**: ajuste `temperature`, `top_p`/`top_k` e **comprimento máximo** para balancear coerência/diversidade.
- **Context window**: limite de tokens do modelo; resumos muito longos podem truncar o início do prompt.
- **Avaliação**: use métrica intrínseca (perplexity) e extrínseca (tarefas específicas); para estilo, considere avaliação humana.

## 8) Notas finais e variações

- **Modelos “GPT-like” em PT-BR**: ao usar um checkpoint treinado em português, espere melhor qualidade sintática/lexical; porém, verifique licença e tamanho do vocabulário.
- **Chat/instruções**: GPT-3.5/4 (e equivalentes open-source com *instruction-tuning*) incluem treinamento adicional com **instruções** e **RLHF**, que melhora “seguir ordens”.
- **Segurança**: adicione filtros de conteúdo e *guardrails* para aplicações reais.
- **Eficiência**: para treinos maiores, use *LoRA/QLoRA*, *gradient checkpointing* e *mixed precision*.

## 9) Exercícios

1. **Temperatura e top-p**: gere 3 variações de um mesmo prompt e compare fluência e factualidade.
2. **Prompt design**: crie *prefixos* (“Instruções: ...”, “Resuma: ...”) e observe o impacto no estilo.
3. **Domínio específico**: substitua o corpus sintético por um conjunto pequeno real (ex.: ementas curtas) e repita o fine-tuning.
4. **Perplexity**: separe um *validation split* e reporte perplexity de validação a cada N passos.